# 1. Import libs

In [1]:
!pip install wandb

In [2]:
import os
import logging
import numpy as np
import random
import matplotlib.pyplot as plt
from itertools import product

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF

from PIL import Image
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision.transforms import v2

from tqdm import tqdm

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("wandb-api-key")

import wandb
wandb.login(key=wandb_api_key)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# 2. Define UNet

In [5]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()

        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.block(x)

In [6]:
class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=19):
        super(UNet, self).__init__()

        # Define channels for different stages
        channels = [64, 128, 256, 512]
        
        # Define pooling layer
        self.maxpool = nn.MaxPool2d(2, 2)
    
        # Define encoder
        self.encoder = nn.ModuleList()
        for channel in channels:
            self.encoder.append(DoubleConv(in_channels, channel))
            in_channels = channel

        # Define bottleneck layer
        self.bottleneck = DoubleConv(channels[-1], channels[-1] * 2)
        
        # Define decoder
        self.decoder = nn.ModuleList()
        for channel in reversed(channels):
            self.decoder.append(nn.ConvTranspose2d(channel * 2, channel, 2, 2))
            self.decoder.append(DoubleConv(channel * 2, channel))

        # Define last layer
        self.last = nn.Conv2d(channels[0], out_channels, 1)

    def forward(self, x):
        # Define storage for connections
        skip_connections = []

        # Down stages
        for block in self.encoder:
            x = block(x)
            skip_connections.append(x)
            x = self.maxpool(x)

        # Bottleneck passing
        x = self.bottleneck(x)

        # Reverse list for easily taking
        skip_connections = skip_connections[::-1]

        # Up stages
        for i in range (0, len(self.decoder), 2):
            # Conv Transpose
            x = self.decoder[i](x)

            # Passing main block
            skip_connection = skip_connections[i // 2]

            ## Resize if neccessary
            if x.shape != skip_connection.shape:
                skip_connection = TF.resize(skip_connection, x.shape[2:]) # Resize except batch and channel
            ## Apply skip connection
            x = torch.cat((skip_connection, x), dim=1) # Concat by channel

            ## Go through conv block
            x = self.decoder[i+1](x)

        # Passing last layer to get output that each channel stand for each instance
        return self.last(x)
                

In [23]:
unet = UNet(3, 19).to(device)
unet = nn.DataParallel(unet)

# 3. Define Dataset and Dataloader

In [8]:
image_size = (512, 1024)

train_transform = v2.Compose([
    v2.Resize((image_size[0], image_size[1]), antialias=True),
    v2.ToTensor(),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


val_transform = v2.Compose([
    v2.Resize((image_size[0], image_size[1]), antialias=True),
    v2.ToTensor(),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [9]:
color_map = np.array([
    [128, 64, 128],   # 0: Road
    [244, 35, 232],   # 1: Sidewalk
    [ 70, 70, 70],    # 2: Building
    [102, 102, 156],  # 3: Wall
    [190, 153, 153],  # 4: Fence
    [153, 153, 153],  # 5: Pole
    [250, 170, 30],   # 6: Traffic light
    [220, 220, 0],    # 7: Traffic sign
    [107, 142, 35],   # 8: Vegetation
    [152, 251, 152],  # 9: Terrain
    [70, 130, 180],   # 10: Sky
    [220, 20, 60],    # 11: Person
    [255, 0, 0],      # 12: Rider
    [0, 0, 142],      # 13: Car
    [0, 0, 70],       # 14: Truck
    [0, 60, 100],     # 15: Bus
    [0, 80, 100],     # 16: Train
    [0, 0, 230],      # 17: Motorcycle
    [119, 11, 32],    # 18: Bicycle
], dtype=np.uint8)

label_mapping = {
    -1: -1,
    0: -1,
    1: -1,
    2: -1,
    3: -1,
    4: -1,
    5: -1,
    6: -1,
    7: 0,
    8: 1,
    9: -1,
    10: -1,
    11: 2,
    12: 3,
    13: 4, 
    14: -1,
    15: -1,
    16: -1,
    17: 5,
    18: -1,
    19: 6,
    20: 7,
    21: 8,
    22: 9,
    23: 10,
    24: 11,
    25: 12,
    26: 13,
    27: 14,
    28: 15,
    29: -1,
    30: -1,
    31: 16,
    32: 17,
    33: 18,
}


In [10]:
random.seed(42)
class CityscapeDataset(Dataset):
    def __init__(self, image_dir, mask_dir, split, transform, flip_prob=0.5):
        self.transform = transform
        self.flip_prob = flip_prob

        # Get all input image paths
        self.image_paths = sorted([os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(image_dir+'/'+split)) for f in fn if f.endswith('.png')])
        self.mask_paths = sorted([os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(mask_dir+'/'+split)) for f in fn if f.endswith('_labelIds.png')])

        if len(self.image_paths) != len(self.mask_paths):
            assert("Warning: Number of images and labels do not match!")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        mask_path = self.mask_paths[idx]

        image = Image.open(image_path).convert('RGB')
        mask = Image.open(mask_path)

        if random.random() < self.flip_prob:
            image = image.transpose(Image.FLIP_LEFT_RIGHT)
            mask = mask.transpose(Image.FLIP_LEFT_RIGHT)
        
        if self.transform is not None:
            image = self.transform(image)
        # image = image.permute(1, 2, 0)
        
        mask = np.array(mask, dtype=np.int64)
        mask = np.vectorize(lambda x: label_mapping.get(x, -1))(mask)
        mask = torch.tensor(mask, dtype=torch.long)

        return image, mask

In [12]:
imgs_dir = "/kaggle/input/cityscapes/Cityspaces/images"
masks_dir = "/kaggle/input/cityscapes/Cityspaces/gtFine"
images = sorted([os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(imgs_dir+'/train')) 
                for f in fn if f.endswith('.png')])
masks = sorted([os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(masks_dir+'/train')) 
                for f in fn if f.endswith('_labelIds.png')])

cnt = 0
for i, m in zip(images, masks):
    if i[49:-16] == m[49:-20]:
        cnt += 1
print(cnt)
assert(cnt == len(images) and cnt == len(masks))

2975


In [13]:
train_dataset = CityscapeDataset(imgs_dir, masks_dir, split='train', transform=train_transform, flip_prob=0.5)
val_dataset = CityscapeDataset(imgs_dir, masks_dir, split='val', transform=val_transform, flip_prob=0)
print(train_dataset[0][0].shape, len(val_dataset))

torch.Size([3, 512, 1024]) 500


In [14]:
def get_sampler(dataset):
    if torch.distributed.is_initialized():
        from torch.utils.data.distributed import DistributedSampler
        return DistributedSampler(dataset)
    else:
        return None

In [15]:
train_subset_indices = list(range(-600, -1))
val_subset_indices = list(range(50))

train_subset = Subset(train_dataset, train_subset_indices)
train_sampler = get_sampler(train_subset)

val_subset = Subset(val_dataset, val_subset_indices)
val_sampler = get_sampler(val_subset)

In [21]:
num_workers = os.cpu_count()
batch_size = 4

train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=num_workers, sampler=train_sampler)
val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, num_workers=num_workers, sampler=val_sampler)

In [39]:
print(len(train_loader), len(val_loader))
img, msk = next(iter(train_loader))
print(img.shape, msk.shape)

150 13
torch.Size([4, 3, 512, 1024]) torch.Size([4, 1024, 2048])


# 4. Define controller

In [16]:
def save_model(model, file_path):
    file_path = file_path + '.pth'
    dir_name = os.path.dirname(file_path)
    if dir_name and not os.path.exists(dir_name):
        os.makedirs(dir_name)
    
    # Save
    torch.save(model.state_dict(), file_path)
    print(f"Model is saved to: {file_path}")

In [24]:
lr = 0.003
WD = 0.001

class_weights = torch.FloatTensor([0.8373, 0.918, 0.866, 1.0345, 
                                        1.0166, 0.9969, 0.9754, 1.0489,
                                        0.8786, 1.0023, 0.9539, 0.9843, 
                                        1.1116, 0.9037, 1.0865, 1.0955, 
                                        1.0865, 1.1529, 1.0507]).cuda()

criterion = nn.CrossEntropyLoss(ignore_index = -1, weight=class_weights)
optimizer = torch.optim.Adam(params = unet.parameters(), lr=lr, weight_decay=WD)
scaler = torch.amp.GradScaler('cuda')

In [42]:
torch.cuda.empty_cache()

In [17]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scaler, 
                device, save_path=None, epochs=10, wandb_writer=None):
    best_loss = float('inf')

    train_losses = []
    val_losses = []
    train_accuracy = []
    val_accuracy = []
    
    model.train()
    for epoch in range(epochs):
        train_loss, train_acc = 0, 0
        logging.info(f"Starting Epoch {epoch + 1}/{epochs}")

        for images, masks in tqdm(train_loader, unit='batch', desc=f'Training [{epoch+1}/{epochs}]'):
            images, masks = images.to(device), masks.to(device)
            
            with torch.amp.autocast('cuda'):
                mask_logits = model(images) # (B, C, H, W)
    
                # Resize to original size of mask
                mask_logits = F.interpolate(mask_logits, size=(1024, 2048), mode='bilinear', align_corners=False)
    
                # # Permute channel to last size
                # mask_logits = mask_logits.permute(0, 2, 3, 1) # (B, H, W, C)
                # print(mask_logits.shape, masks.shape)
                # Loss
                loss = criterion(mask_logits, masks.squeeze(1))
            train_loss += loss.item()

            # Update
            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            # Calculate accuracy over pixel
            mask_pred = torch.argmax(mask_logits, dim = 1)
            mask_pred[masks == -1] = -1
            
            valid_mask = masks != -1
            acc_per_batch = torch.eq(mask_pred[valid_mask], masks[valid_mask]).sum().item()
            train_acc += (acc_per_batch / masks.numel())
        avg_train_loss = train_loss / len(train_loader)
        avg_train_acc = train_acc / len(train_loader)
        
        print(f'---> Train loss: {avg_train_loss:.4f}')
        print(f'---> Train acc: {avg_train_acc:.4f}')

        train_losses.append(avg_train_loss)
        train_accuracy.append(avg_train_acc)
        if wandb_writer is not None:
            wandb.log({"Train_loss": avg_train_loss, "Train_acc": avg_train_acc})
        
        # Validate
        val_loss, val_acc = 0,0
        model.eval()
        with torch.inference_mode():
            for images, masks in tqdm(val_loader, unit='batch', desc=f'Validate [{epoch+1}/{epochs}]'):
                images, masks = images.to(device), masks.to(device)
                
                with torch.amp.autocast('cuda'):
                    mask_logits = model(images)
                    mask_logits = F.interpolate(mask_logits, size=(1024, 2048), mode='bilinear', align_corners=False)
                    # mask_logits = mask_logits.permute(0,2,3,1)
                    loss = criterion(mask_logits, masks.squeeze(1))
                val_loss += loss.item()
                
                mask_pred = torch.argmax(mask_logits, dim = 1)
                mask_pred[masks == -1] = -1

                valid_mask = masks != -1
                acc_per_batch = torch.eq(mask_pred[valid_mask], masks[valid_mask]).sum().item()
                val_acc += (acc_per_batch / masks.numel())
        avg_val_loss = val_loss / len(val_loader)
        avg_val_acc = val_acc / len(val_loader)
        
        print(f'---> Valid loss: {avg_val_loss:.4f}')
        print(f'---> Valid acc: {avg_val_acc:.4f}')
        
        val_losses.append(avg_val_loss)
        val_accuracy.append(avg_val_acc)
        if wandb_writer is not None:
            wandb.log({"val_loss": avg_val_loss, "val_acc": avg_val_acc})
        if save_path is not None:
            if avg_val_loss < best_loss:
                best_loss = avg_val_loss
                save_model(model, save_path + f'_epoch{epoch+1}')
                if wandb_writer is not None:
                    wandb.save(f"{save_path}_epoch{epoch+1}.pth")
    return train_losses, val_losses, train_accuracy, val_accuracy

# 5. Hyperparameter tuning

In [39]:
hyper_params = {
    'lr': [0.003, 0.001, 0.0005, 0.0003],
    'weight_decay': [0.01, 0.001],
    'optim': ['Adam', 'SGD']
}

In [40]:
def hyper_tuning_model(model_class, hyper_params, train_loader, val_loader, 
                       device, save_path, class_weights, epochs=10):

    wandb.init(project="unet-hyperparam-tuning", reinit=True)
    
    keys, values = hyper_params.keys(), hyper_params.values()
    param_combinations = [dict(zip(keys, v)) for v in product(*values)]

    best_params = None
    best_accuracy = 0.0

    for params in param_combinations:
        wandb.config.update(params)
        print(f"Training with parameters: {params}")
        
        # Clone the model to ensure it's fresh for each configuration
        current_model = model_class(3, 19).to(device)
        current_model = nn.DataParallel(current_model)
        # Define optimizer
        if params['optim'] == 'Adam':
            optimizer = torch.optim.Adam(current_model.parameters(), lr=params['lr'], weight_decay=params['weight_decay'])
        elif params['optim'] == 'SGD':
            optimizer = torch.optim.SGD(current_model.parameters(), lr=params['lr'], weight_decay=params['weight_decay'])
        
        criterion = nn.CrossEntropyLoss(ignore_index=-1, weight=class_weights)
        scaler = torch.amp.GradScaler('cuda')
        # Training loop
        train_losses, val_losses, train_accuracy, val_accuracy = train_model(current_model, train_loader, val_loader, 
                                                                             criterion, optimizer, scaler, 
                                                                             device, save_path, 
                                                                             epochs=epochs, wandb_writer=wandb)

        # Check if this configuration is the best
        if np.max(val_accuracy) > best_accuracy:
            best_accuracy = np.max(val_accuracy)
            best_params = params
            model_path = f"{save_path}_{params['lr']}_{params['weight_decay']}_{params['optim']}"
            save_model(current_model, model_path)
            artifact = wandb.Artifact(f"unet-model-{params['lr']}-{params['weight_decay']}-{params['optim']}", type="model")
            artifact.add_file(model_path + '.pth')
            wandb.log_artifact(artifact)
    
    wandb.finish()
    return {
        'best_params': best_params,
        'best_accuracy': best_accuracy
    }

In [41]:
save_path = '/kaggle/working/tuned/unet'
best_result = hyper_tuning_model(UNet, hyper_params, train_loader, val_loader, 
                                 device, save_path, class_weights, epochs=10)

# Kết quả tốt nhất
print(f"Best Hyperparameters: {best_result['best_params']}")
print(f"Best Validation Accuracy: {best_result['best_accuracy']:.4f}")

Train_acc,█▁▅▇▇▇██
Train_loss,▄█▄▂▃▂▂▁
val_acc,▄▆▆▁▆▇▆█
val_loss,▆▄▆█▄▃▃▁
Train_acc,0.55073
Train_loss,1.32804
val_acc,0.55344
val_loss,1.29323


Training with parameters: {'lr': 0.003, 'weight_decay': 0.01, 'optim': 'Adam'}


Training [1/10]: 100%|██████████| 150/150 [02:43<00:00,  1.09s/batch]


---> Train loss: 1.4194
---> Train acc: 0.5431


Validate [1/10]: 100%|██████████| 13/13 [00:13<00:00,  1.04s/batch]


...

---> Train loss: 1.1890
---> Train acc: 0.5973


Validate [10/10]: 100%|██████████| 13/13 [00:12<00:00,  1.03batch/s]


---> Valid loss: 1.2874
---> Valid acc: 0.5520


Train_acc,▅▅▆▆▆▅▆▆▆▆▅▆▆▆▇▆▃▂▂▂▆▆▇▄▄▆▆▅▇▇▅▆▇███▁▆▆▆
Train_loss,▃▃▃▂▂▃▃▃▃▂▂▂▃▃▃▂▃▂▂█ ▅▃▂ ▃▂▂▂▂▁▁▁▁▃
val_acc,▆▆▆▆▆█▆▇▅▇▇▇▆▆▆▇▆▇▇▇▁▄▃▃▃▆▅▆▆▇▇▇█▅▆█▇█▆▇
val_loss,▆▅▅▆▅▄▄▅▅▆▅▄▄▃▃▃ ▄▅█ ▆▃▄▂▂█▅▅▄▄▂▂▁▁▄
Train_acc,0.59731
Train_loss,1.18901
val_acc,0.55197
val_loss,1.28742


Best Hyperparameters: {'lr': 0.0003, 'weight_decay': 0.001, 'optim': 'Adam'}
Best Validation Accuracy: 0.6511


In [42]:
torch.save(best_result['best_params'], '/kaggle/working/tuned/best_params_1.pth')

In [46]:
wandb.init(project="unet-hyperparam-tuning")

In [47]:
artifact = wandb.Artifact(f"unet-hyper-params", type="hyperparams")
artifact.add_file('/kaggle/working/tuned/best_params_1.pth')
wandb.log_artifact(artifact)

<Artifact unet-hyper-params>

In [48]:
wandb.finish()